In [1]:
# General Imports
import pandas as pd
import sys
import os
import re
import pickle

# Map src directory
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../."))
print("Root Directory: ", root_dir)
src_dir = os.path.join(root_dir,"src")
print("Src Directory: ", src_dir)
sys.path.append(src_dir)
data_dir = os.path.join(root_dir,"data")
print("Data Directory: ", data_dir)

# Import src modules
import complete_collision as cc
import utils as u

Root Directory:  /media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder
Src Directory:  /media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/src
Data Directory:  /media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/data


/media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
training_df = pd.read_csv(os.path.join(data_dir, "processed", "training_df.csv"))
training_df['concatenated_text'] = training_df.apply(u.concatenate_texts, axis=1)
u.clean_text(training_df, 'concatenated_text')

,CAD_ID,CAD_TEXT,OH1_TEXT,BIKE_CLE_TEXT,CRASH_SEVERITY_CD,INTERSECTION_IND,U1_PRECRASH_ACTION_CD,U1_TURN_CD,U1_TYPE_OF_UNIT_CD,U2_PRECRASH_ACTION_CD,U2_TURN_CD,U2_TYPE_OF_UNIT_CD,concatenated_text
0,202000000930,INCIDENT REPORT PRINT DATE/TIME: 05/20/2024 10...,NaN,A driver struck an adult crossing an intersect...,3.0,Y,99.0,NT,1.0,1.0,NT,1.0,CALL COMMENTS COMMENTS FROM INTERGAPH 1/1/2020...
1,202000001607,NaN,"WHILE UNIT1 WAS TURNING ONTO W44_, PEDESTRIAN ...",A driver of a commercial truck turned left and...,5.0,Y,1.0,NT,1.0,11.0,NT,1.0,POLICE NARRATIVE \n\n WHILE UNIT1 WAS TURNING ...
2,202000002596,INCIDENT REPORT PRINT DATE/TIME: 05/20/2024 10...,NaN,A driver struck a person. The crash caused inj...,2.0,Y,6.0,LT,3.0,1.0,NT,1.0,CALL COMMENTS COMMENTS FROM INTERGAPH 1/3/2020...
3,202000002644,INCIDENT REPORT PRINT DATE/TIME: 05/20/2024 10...,(JNIT 1 IS S/B ON E.99 STREET TURNING E/B (LEF...,A driver turned left and struck and killed an ...,5.0,Y,1.0,NT,99.0,11.0,NT,1.0,CALL COMMENTS COMMENTS FROM INTERGAPH 1/3/2020...
4,202000003457,INCIDENT REPORT PRINT DATE/TIME: 05/20/2024 10...,NaN,A driver struck an adult. The crash caused inj...,5.0,Y,1.0,NT,1.0,NaN,NaN,NaN,CALL COMMENTS COMMENTS FROM INTERGAPH 1/4/2020...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,202300386537,PRINT DATE/TIME: LOGIN ID: INCIDENT DATE/TIME:...,UNKNOWN EVENTS,A driver of an SUV turned left and struck an a...,4.0,Y,6.0,LT,99.0,14.0,NT,23.0,PRINT DATE/TIME: LOGIN ID: INCIDENT DATE/TIME:...
2170,202300386861,INCIDENT REPORT PRINT DATE/TIME: 01/02/2024 10...,NaN,A driver of a transit bus struck a person.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CALL COMMENTS COMMENTS FROM INTERGAPH 12/30/20...
2171,202300387472,PRINT DATE/TIME: 01/05/2024 06:37 LOGIN ID: GR...,NaN,A driver struck a person.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRINT DATE/TIME: 01/05/2024 06:37 LOGIN ID: GR...
2172,202300388004,PRINT DATE/TIME: LOGIN ID: VECCHIOS INCIDENT: ...,NaN,A driver struck a person on a bicycle. The dri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRINT DATE/TIME: LOGIN ID: VECCHIOS INCIDENT: ...


In [3]:
with open('severity.pkl', 'rb') as f:
    severity_model = pickle.load(f)

training_df['SEVERITY_PRED'] = severity_model.predict(training_df['concatenated_text'])
training_df['SEVERITY_PRED'].value_counts()

SEVERITY_PRED
0    1894
1     280
Name: count, dtype: int64